# Question 2

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge, RidgeCV, LassoCV, lasso_path, ElasticNetCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In this question, I will, as instructed, be testing different forms of regularization and its effect on regression. For this specific question, we will be regressing index of democracy on various factors, such as GDP per capita, population size, age breakdown of population, and average years of education of the population. The final goal is to ultimately see, with regularization, if there is a true relationship between GDP per capita and democracy, with other variables included to soak up standard error and reduce any potential omission bias, with the hypothesis being that these other variables, which are not necessarily the main factors of study, are assumed to potentially be correlated with democracy index. While standard regression is almost forced to assign a coefficient to each regressor, these regularization methods allow the model to deflate the coefficients, or even reduce to 0 depending on the model, thus shining more light on which variables might truly be the most impactful in terms of affecting the dependent variable. Thus, with these methods and other regressors, I hope to see, while practicing various regularization methods, if GDP per capita truly is the most important factor, or is steadily included with a non-zero coefficient for all modeling methods.

In [2]:
df = pd.read_csv(r"../income_democracy.csv")
scaler = StandardScaler()
df = df.dropna()
yy = df['dem_ind']

df = df.drop(['code','country','year','dem_ind'],axis=1)

df[df.columns] = scaler.fit_transform(df[df.columns])

x_train, x_test, y_train, y_test = train_test_split(df,yy,random_state=1993, train_size=.8, test_size=.2)

In the above cell, I load the data, drop the columns we are told to drop in Piazza, drop any rows with null data, scale the data (which is a necessary step for regularization methods), and finally split the data into train and test sets. 

# Ridge

As stated in lecture, ridge regression serves to reduce the coefficients. While it cannot perform variable selection, ridge still serves to decrease final variance by adding some bias into the model, and relaxes the assumption with standard linear regression that regressors must be unbiased -- with ridge, we can use biased regressors, and with such will get better results than standard regression. Furthermore, it helps best in situations in which many regressors have similar-magnitude impacts on a dependent variable. 

In [3]:
ridge = RidgeCV(alphas = list(np.arange(1,50,.1)), cv=10).fit(x_train,y_train)

Above, I use the all-in-one function from sklearn called RidgeCV, and test various values of alpha from 1 to 50, in 0.1 intervals, via cross-validation. I then fit the model with the training data, which then tunes the alpha value as it trains. 

In [4]:
print(f"Final Optimal Alpha for Ridge: {round(ridge.alpha_,3)}")

Final Optimal Alpha for Ridge: 39.6


In [5]:
ridge_preds = ridge.predict(X = x_test)
print(f"MSE for Test Prediction for Ridge: {round(mean_squared_error(y_test,ridge_preds),3)}")

MSE for Test Prediction for Ridge: 0.054


In [6]:
coefs_ridge = pd.DataFrame([df.columns,ridge.coef_])
print("The following table includes the coefficients for the Regressors with Ridge Regression")
coefs_ridge.T.rename({0:'Variable',1:'Coefficient'},axis=1).set_index('Variable')

The following table includes the coefficients for the Regressors with Ridge Regression


,Coefficient
Variable,
log_gdppc,0.099319
log_pop,-0.011793
age_1,-0.022159
age_2,0.003654
age_3,-0.020758
age_4,0.028502
age_5,0.030803
educ,0.079497
age_median,0.016462


Looking at the coefficients, we can see that the log-GDP has one of the lowest coefficients in absolute value across all coefficients, which is interesting. Notice also how age median is not given a coefficient, perhaps due to collinearity issues with the other age variables. 

# Lasso

Now, moving on to Lasso: Lasso is best at variable selection, in that it is able to reduce certain coefficients to 0 due to its penalty term. It  also adds a bit of bias into the model in order to reduce variance, and allows for using regressors that might be correlated, as Lasso reduces the coefficients in a way to account for any correlation or possible covariates. 

In [7]:
lasso = LassoCV(alphas = list(np.arange(0.001,2,.001)), cv=10, max_iter=10000).fit(x_train,y_train)
print(f"Final Optimal Alpha for Lasso: {round(lasso.alpha_,3)}")

Final Optimal Alpha for Lasso: 0.006


Again, above I use the all-in-one cross-validation function for Lasso, in which I try various levels of alpha, ranging from .001 to 2, in intervals of .001. The optimal alpha is printed above, and was used in final fitting. 

In [8]:
lasso_preds = lasso.predict(X = x_test)
print(f"MSE for Test Prediction for Lasso: {round(mean_squared_error(y_test,lasso_preds),3)}")

MSE for Test Prediction for Lasso: 0.054


In [9]:
coefs_lasso = pd.DataFrame([df.columns,lasso.coef_])
print("The following table includes the coefficients for the Regressors with Lasso Regression")
coefs_lasso.T.rename({0:'Variable',1:'Coefficient'},axis=1).set_index('Variable')

The following table includes the coefficients for the Regressors with Lasso Regression


,Coefficient
Variable,
log_gdppc,0.112588
log_pop,-0.005378
age_1,-0.0
age_2,-0.0
age_3,-0.0
age_4,0.021004
age_5,0.042281
educ,0.07971
age_median,0.0


Notice first how, with Lasso, we do have four variables whose coefficients drop to 0: Ages1 to 3 and Age_median! Thus, it shows that perhaps a lot of the age columns were redundant, and now GDP has the largets absolute effect! The MSE on test set is nearly identical to that of ridge. 

HOWEVER, one flaw with Lasso is that, if there ARE covariates or correlated variables in the regressors, Lasso does NOT necessarily have a reason for "choosing" one variable over the other, and thus might not always tell the "correct" story. Thus, it is not clear if ages 1-3 or age_median are truly the redundant variables, or if they, in place of age 4 and 5, would be "better predictors".

# Adaptive Lasso

Adaptive Lasso is a version of lasso regularization, but in which the parameters are weighted, thus reducing a common effect of regular LASSO in which larger coefficients are shrunk to a larger extent than smaller coefficients, thus overly biasing the model. With adaptive lasso, the larger coefficients are reduced less than their regular lasso counterparts, compared to regressors with smaller coefficients. 

In [10]:
gamma = 1
p=len(x_train.columns)
ridge_betas = RidgeCV(
    fit_intercept=True, alphas=np.logspace(-6, 6, num=100)).fit(x_train, y_train).coef_
w_ridge = ridge_betas**-gamma
X_ridge = x_train/w_ridge
lambdas, lasso_betas, _ = lasso_path(X_ridge, y_train)

lasso_betas = lasso_betas/w_ridge[:, None]
lasso_coef = pd.DataFrame(index=lambdas, data=lasso_betas.T)
lasso_coef.columns = [f'B{i}' for i in range(1, p+1)]
non_zero = lasso_coef.abs().mean() > 1e-1
lasso_coef = lasso_coef.loc[:, non_zero]

lassoCV = LassoCV(alphas=lambdas, fit_intercept=True, cv=10)
lassoCV.fit(X_ridge, y_train)  
lassoMSEs = pd.Series(lassoCV.mse_path_.mean(1), index=lambdas)
lambda_lasso = lassoMSEs.idxmin()
lasso_score = np.mean((lassoCV.predict(X_ridge)-y_train)**2)
print(f'Adaptive Lasso Optimal lambda = {lambda_lasso}')

Adaptive Lasso Optimal lambda = 6.137342527177675e-05


In the above cell, I set up the adaptive lasso algorithm as given in the sample code, this time with gamma = 1. Please see print-out for optimal Lambda value.

After setting up and fitting the adaptive lasso model, I then test it on test data below: 

In [11]:
X_ridge_test = x_test/w_ridge
adaptive_preds = lassoCV.predict(X_ridge_test)
print(f"MSE for Test Prediction for ADAPTIVE Lasso: {round(mean_squared_error(y_test,adaptive_preds),3)}")

MSE for Test Prediction for ADAPTIVE Lasso: 0.053


As can be seen, the MSE went down ever so slight with adaptive Lasso vs regular Lasso (by .003 points, which is very minor, if even significant). 

In [12]:
coefs_adaptive = pd.DataFrame([df.columns,lassoCV.coef_])
print("The following table includes the coefficients for the Regressors with ADAPTIVE Lasso Regression")
coefs_adaptive.T.rename({0:'Variable',1:'Coefficient'},axis=1).set_index('Variable')

The following table includes the coefficients for the Regressors with ADAPTIVE Lasso Regression


,Coefficient
Variable,
log_gdppc,1.174748
log_pop,0.409131
age_1,0.0
age_2,0.0
age_3,0.599415
age_4,1.034946
age_5,1.202417
educ,1.069954
age_median,0.0


As can be seen, with adaptive lasso, only 3 coefficients got reduced to 0: Ages 1, 4, and 5 are now with 0-coefficient, and age_median and age_2 show up, which could be due to how lasso sometimes randomly decides which coefficients to reduce to 0 and which to keep. Oddly enough, in this run, age_3 and age_median have the highest coefficients.

# Elastic Net

As the final model, I will run an elastic net model, which is a form of regularization in which there is a balanced L1 and L2 penalty term (A fraction of penalty term is from L1, and the other fraction from L2). Thus, with elastic net, not only do we have to tune lambda, but also a new parameter, which serves as the ratio of L1/L2 punishment. 

Luckily, there is a handy-dandy function in sklearn that does the dual optimization for me using cross-validation: ElasticNetCV. I pass in a list of possible values for the lambda term and the l1-ratio term, which I picked based on prior runs to reduce run-time. 

In [13]:
elastic = ElasticNetCV(l1_ratio = list(np.arange(0.1,1.001,.01)), alphas = list(np.arange(.001,1,.001)), cv=10, max_iter=100000).fit(x_train,y_train)
print(f"Optimal lambda for Elastic Net: {elastic.alpha_} ... Optimal l1/l2 penalization ratio: {round(elastic.l1_ratio_,3)}")

Optimal lambda for Elastic Net: 0.041 ... Optimal l1/l2 penalization ratio: 0.12


Notice that the ratio is higher-weighted toward l2, meaning that the model benefits from higher ridge-like penalty than lasso! 

In [14]:
elastic_preds = elastic.predict(X = x_test)
print(f"MSE for Test Prediction for Elastic Net: {round(mean_squared_error(y_test,elastic_preds),3)}")

MSE for Test Prediction for Elastic Net: 0.054


This model, has a similar MSE as ridge and lasso.

In [15]:
coefs_elastic = pd.DataFrame([df.columns,elastic.coef_])
print("The following table includes the coefficients for the Regressors with Elastic Net")
coefs_elastic.T.rename({0:'Variable',1:'Coefficient'},axis=1).set_index('Variable')

The following table includes the coefficients for the Regressors with Elastic Net


,Coefficient
Variable,
log_gdppc,0.105273
log_pop,-0.006922
age_1,-0.000783
age_2,0.0
age_3,-0.0
age_4,0.026351
age_5,0.041927
educ,0.079991
age_median,0.0


Finally, looking at the coefficients, we can see again that some age variables get reduced to 0 (age 2,3, and median), while gdp again has the highest absolute value coefficient of all. 

# Conclusion

In conclusion, each model seems to have its advantages and disadvantages, and thus to answer the question of which model I would ultimately choose, I will approach that question in 2 ways: (1) the model that ultimately seems to best predict democracy coefficient, and then (2) the model that might have the best indicator of the effect of GDP on democracy (which might arguably be the same model). 


In terms of simply picking the best model, My criteria for picking a model are that the model obviously performs well, and also, when performance is similar, I would pick the simpler model, or that which ultimately has less regressors with non-zero coefficients. 

Thus, in terms of model accuracy, all models perform nearly identically, with adaptive lasso being the only model that gets a slightly better MSE. Thus, since performance is similar across all methods, I would choose a lasso-based method, due to the their simplicity. Since basic lasso was able to 0-out 4 variables, I might pick it as my final model just in terms of accuracy + simplicity.

However, given that **adaptive lasso has the best MSE, and still pushes 3 variables to 0-coefficient, I might choose adaptive lasso as the best model overall**. Furthermore, the concept of adaptive lasso lends somewhat better to the task at hand, which is ultimately understanding the effect that GDP has on democracy. With regular lasso, bigger coefficients get reduced more in magnitude, and thus with adaptive, we get a better sense of perhaps the "true" effect of a variable, giving more "explanatory power" to the results.